<a href="https://colab.research.google.com/github/giemmecci/IDC/blob/main/TCGA_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some of the cohorts in the IDC have multiple studies for each subject.

For example, for some of its subjects, the TCGA-GBM cohort contains both pre-surgery and post-surgery MRI data.

It is important to select the correct study for a given task.

In this notebook, we will select the earliest study for each subject leveraging the StudyDate information stored in the DICOM metadata.

# Before we start:
- Specify the project ID for billing purposes.
- Authenticate to the Google account connected to the project (grants access to the data) 

In [1]:
myProjectID="idc-external-005"

In [2]:
from google.colab import auth
auth.authenticate_user()

# Utilities

In [3]:
# This will make pandas dataframes interactive

# Uncomment next line to activate
%load_ext google.colab.data_table

# Uncomment next line to deactivate
# %unload_ext google.colab.data_table

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Import MRI data from the TCGA-GBM cohort

In [20]:
%%bigquery --project=$myProjectID tcga_gbm

WITH
  list_gbm AS (
  SELECT
    DISTINCT(StudyInstanceUID),
    StudyDate,
    PatientID
  FROM
    `canceridc-data.idc_views.dicom_all`
  WHERE
    collection_id = "tcga_gbm"
    AND
    Modality = 'MR')
  SELECT
    PatientID,
    StudyDate,
    StudyInstanceUID
  FROM
    list_gbm

Let's check how many cases and how many studies we have

In [21]:
num_subjects = len(tcga_gbm["PatientID"].unique())
num_studies = len(tcga_gbm["StudyInstanceUID"].unique())
print(f"Total number of subjects: {num_subjects}")
print(f"Total number of studies: {num_studies}")


Total number of subjects: 259
Total number of studies: 531


In [22]:
# Let's see what we got. Click "Filter" to interact with the dataframe

tcga_gbm

,PatientID,StudyDate,StudyInstanceUID
0,TCGA-06-0125,2001-11-05,1.3.6.1.4.1.14519.5.2.1.4591.4001.287498805432295459381457059707
1,TCGA-14-3477,2003-02-19,1.3.6.1.4.1.14519.5.2.1.2783.4001.111231080354945828079688635322
2,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239
3,TCGA-14-0789,1998-03-05,1.3.6.1.4.1.14519.5.2.1.2783.4001.117194620147640347974163614816
4,TCGA-14-1794,1998-04-11,1.3.6.1.4.1.14519.5.2.1.2783.4001.867649433395377496837912142545
...,...,...,...
526,TCGA-14-1795,1998-07-26,1.3.6.1.4.1.14519.5.2.1.2783.4001.225830234488613134729542421521
527,TCGA-06-0185,2003-11-07,1.3.6.1.4.1.14519.5.2.1.4591.4001.232938998404177432213094426945
528,TCGA-12-3650,2001-07-29,1.3.6.1.4.1.14519.5.2.1.8862.4001.853973730099261399255070895330
529,TCGA-14-1402,1999-09-03,1.3.6.1.4.1.14519.5.2.1.2783.4001.547699000925909667108557240765


## In the following cells we'll select only the earliest study for each subject.

In [23]:
# Here we use sorting and drop duplicates capabilities of pandas

filter_df = tcga_gbm.sort_values('StudyDate').drop_duplicates(subset=['PatientID'], keep='first')

In [24]:
filter_df

,PatientID,StudyDate,StudyInstanceUID
427,TCGA-27-2527,1986-10-11,1.3.6.1.4.1.14519.5.2.1.3775.4001.758562469200608426293731071579
134,TCGA-27-2518,1987-05-24,1.3.6.1.4.1.14519.5.2.1.3775.4001.977371577534469283226269099172
272,TCGA-27-2521,1987-07-09,1.3.6.1.4.1.14519.5.2.1.3775.4001.119470023222804500108388734458
191,TCGA-27-2524,1987-09-06,1.3.6.1.4.1.14519.5.2.1.3775.4001.187020697807720182756891883983
90,TCGA-27-2523,1987-09-27,1.3.6.1.4.1.14519.5.2.1.3775.4001.220256716193578624337019877684
...,...,...,...
238,TCGA-06-5412,2008-06-03,1.3.6.1.4.1.14519.5.2.1.4591.4001.221248717333016631625848271272
160,TCGA-06-5413,2008-06-17,1.3.6.1.4.1.14519.5.2.1.4591.4001.397375653956580386291577018002
419,TCGA-06-5417,2008-09-03,1.3.6.1.4.1.14519.5.2.1.4591.4001.304604545029494418165835320551
319,TCGA-06-6389,2009-04-04,1.3.6.1.4.1.14519.5.2.1.4591.4001.296868957331552138094328434431


At this point, filter_df contains one StudyID per subject, selected based on the earliest study date.

We will use this list to filter the full dataframe and keep only the DICOM files that matches the StudyID.

In [28]:
studyID = filter_df['StudyInstanceUID']
studyID_list = studyID.to_list()

In [29]:
len(studyID_list)

259

In [30]:
filtered_df = tcga_gbm[tcga_gbm['StudyInstanceUID'].isin(studyID_list)]

In [31]:
filtered_df

,PatientID,StudyDate,StudyInstanceUID
2,TCGA-02-0003,1997-06-08,1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239
4,TCGA-14-1794,1998-04-11,1.3.6.1.4.1.14519.5.2.1.2783.4001.867649433395377496837912142545
6,TCGA-14-0783,1992-08-16,1.3.6.1.4.1.14519.5.2.1.2783.4001.128196123307003215321475518110
9,TCGA-27-1837,1989-08-23,1.3.6.1.4.1.14519.5.2.1.3775.4001.242002842856501312196534228391
10,TCGA-27-2519,1987-11-15,1.3.6.1.4.1.14519.5.2.1.3775.4001.255418773275260628805882964864
...,...,...,...
523,TCGA-19-1386,2001-08-15,1.3.6.1.4.1.14519.5.2.1.5826.4001.107011024013054365393272708356
525,TCGA-19-5956,2004-01-27,1.3.6.1.4.1.14519.5.2.1.5826.4001.193530958293483695173623699128
526,TCGA-14-1795,1998-07-26,1.3.6.1.4.1.14519.5.2.1.2783.4001.225830234488613134729542421521
527,TCGA-06-0185,2003-11-07,1.3.6.1.4.1.14519.5.2.1.4591.4001.232938998404177432213094426945


At this point, we should have one study per subject.

In [32]:
# Check that we did not loose any subject in the process

num_filtered_subjects = len(filtered_df["PatientID"].unique())
print(f"Total number of subjects: {num_filtered_subjects}")
num_filtered_studies = len(filtered_df["StudyInstanceUID"].unique())
print(f"Total number of studies: {num_filtered_studies}")

Total number of subjects: 259
Total number of studies: 259


# Now we can download the data we are interested in.

To keep it simple, we'll select only the first two subject.

In [35]:
to_download = []
base_gs_uri = 'gs://idc-tcia-tcga-gbm/dicom/'

for i in range(2):
    to_download.append(base_gs_uri + filtered_df.iloc[i]['StudyInstanceUID'])

download_df = pd.DataFrame({"gcs":to_download},columns=["gcs"])

In [36]:
download_df

,gcs
0,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239
1,gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.2783.4001.867649433395377496837912142545


In [37]:
download_df.to_csv("gcs_paths.txt",header=False, index=False)
!head gcs_paths.txt

gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4001.145725991542758792340793681239
gs://idc-tcia-tcga-gbm/dicom/1.3.6.1.4.1.14519.5.2.1.2783.4001.867649433395377496837912142545


In [ ]:
!mkdir downloaded_cohort
!cat gcs_paths.txt | gsutil -u $myProjectID -m cp -Ir ./downloaded_cohort

In [ ]:
!git clone https://github.com/pieper/dicomsort.git
!pip install pydicom
!python dicomsort/dicomsort.py --help

In [40]:
!python dicomsort/dicomsort.py -u downloaded_cohort cohort_sorted/%PatientID/%StudyDate/%StudyInstanceUID/%SeriesDescription/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 899/899 [00:02<00:00, 336.73it/s]
Files sorted


In [27]:
!python dicomsort/dicomsort.py -u downloaded_cohort cohort_sorted_2/%PatientID/%StudyDate/%StudyInstanceUID/%SeriesDescription/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 1272/1272 [00:05<00:00, 247.50it/s]
Files sorted


## DEV

In [ ]:
%%bigquery --project=$myProjectID cohort_df

SELECT
    DISTINCT(PatientID),
    StudyInstanceUID,
    StudyDate,
    gcs_url
FROM `idc-external-005.test_gm.tcga-gbm-mr`
LIMIT 20

In [ ]:
cohort_df

In [ ]:
num_subjects = len(cohort_df["PatientID"].unique())
num_studies = len(cohort_df["StudyInstanceUID"].unique())
print(f"Total number of subjects: {num_subjects}")
print(f"Total number of studies: {num_studies}")

Total number of subjects: 259
Total number of studies: 531
